In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
from google.colab import files

In [35]:
pd.options.display.float_format = "{:.6g}".format

In [70]:
uploaded = files.upload()
uploaded_paths = list(uploaded.keys())
print("Uploaded files:", uploaded_paths)

Saving 202501_기준금속.xlsx to 202501_기준금속 (1).xlsx
Uploaded files: ['202501_기준금속 (1).xlsx']


In [36]:
# 자동 검출된 컬럼명 확인 후 True 반영
CONFIRM_DETECTED_COLUMNS = True

# 전처리 옵션
NEGATIVE_TO_NAN = True   # 음수값을 NaN으로 처리
USE_IQR_OUTLIER_FILTER = True  # IQR 기반 이상치 제거 적용

# IQR 필터 강도(기본 1.5)
IQR_K = 1.5

# 시각화 및 리포트 파일명
RAW_REPORT_PATH   = "Step1_QA_Report_raw.xlsx"    # 전처리 전 QA 리포트
CLEAN_REPORT_PATH = "Step1_QA_Report_clean.xlsx"  # 전처리 후 QA 리포트
CLEAN_DATA_PATH   = "Cleaned_dataset.xlsx"        # 전처리 후 데이터(엑셀)
CLEAN_DATA_CSV    = "Cleaned_dataset.csv"         # 전처리 후 데이터(CSV)

In [37]:
# 컬럼 자동검출 함수
def detect_columns(df: pd.DataFrame):
    # 공백 제거
    df = df.copy()
    df.columns = [str(c).strip() for c in df.columns]

    # 시간 컬럼
    time_candidates = [c for c in df.columns
                       if any(k in c.lower() for k in ["pump-begin", "date", "datetime", "time", "측정일시"])]
    time_col = time_candidates[0] if len(time_candidates) > 0 else None

    # PM2.5 컬럼
    pm25_candidates = [c for c in df.columns
                       if ("pm2.5" in c.lower())
                       or ("con(ug/m3)" in c.lower())
                       or ("ug/m3" in c.lower() and "con" in c.lower())]
    pm25_col = pm25_candidates[0] if len(pm25_candidates) > 0 else None

    # 금속 단위 패턴
    metal_markers = ["(ng/m3)", "(ng/㎥)", "ng/m3", "ng/㎥"]
    metal_cols = [c for c in df.columns if any(m.lower() in c.lower() for m in metal_markers)]

    # 숫자형 후보
    numeric_cols = []
    for c in df.columns:
        ser = pd.to_numeric(df[c], errors="coerce")
        if ser.notna().mean() >= 0.7:
            numeric_cols.append(c)

    return {
        "time_col": time_col,
        "pm25_col": pm25_col,
        "metal_cols": metal_cols,
        "numeric_cols": numeric_cols}

In [38]:
# 컬럼 자동 검출 및 확인게이트
path = uploaded_paths[0]

df_raw = pd.read_excel(path)
df_raw.columns = [str(c).strip() for c in df_raw.columns]

# 자동 검출
detected = detect_columns(df_raw)
print("Detected time_col:", detected["time_col"])
print("Detected pm25_col:", detected["pm25_col"])
print("Detected metal_cols (first 10):", detected["metal_cols"][:10])
print("Detected numeric_cols (first 10):", detected["numeric_cols"][:10])

# 자동 검출 컬럼 확인
if not CONFIRM_DETECTED_COLUMNS:
    raise RuntimeError(
        "자동 검출된 컬럼 확인 후 맞다면 CONFIRM_DETECTED_COLUMNS=True로 변경 후 이 셀부터 다시 실행")

# 시간 컬럼 파싱
time_col = detected["time_col"]
pm25_col = detected["pm25_col"]
metal_cols = detected["metal_cols"]
numeric_cols = detected["numeric_cols"]

if time_col is not None:
    df_raw[time_col] = pd.to_datetime(df_raw[time_col], errors="coerce")

Detected time_col: Pump-Begin
Detected pm25_col: Conc(ug/m3)
Detected metal_cols (first 10): ['Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)', 'Sb(ng/m3)', 'Pb(ng/m3)']
Detected numeric_cols (first 10): ['Pump-Begin', 'Pump-End', 'MassResetTime', 'Conc(ug/m3)', 'Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)', 'Sb(ng/m3)']


In [69]:
# 전처리 전 설정
NEGATIVE_TO_NAN   = True   # 음수값을 NaN으로 치환
DROP_EMPTY_TARGET = True   # 대상 컬럼이 모두 NaN인 행 제거
APPLY_IQR_FILTER  = True   # IQR 기반 이상치 제거 적용
IQR_K             = 1.5    # IQR 배수(1.5 기본, 완화 2.0~3.0)

In [67]:
# 전제 조건
assert "df_raw" in globals(), "df_raw가 필요합니다."

# 대상 컬럼 정의 (PM2.5 + 금속)
target_cols = []
if "pm25_col" in globals() and pm25_col:
    target_cols.append(pm25_col)
if "metal_cols" in globals() and isinstance(metal_cols, (list, tuple)):
    target_cols += list(metal_cols)

# 실제 존재하는 컬럼만 남기기
target_cols = [c for c in target_cols if c in df_raw.columns]
if len(target_cols) == 0:
    raise ValueError("대상 컬럼을 찾지 못했습니다.")

# 복사본 생성
df_clean = df_raw.copy()

In [66]:
# 대상 컬럼을 숫자로 변환한 프레임 준비
num_targets = df_clean[target_cols].apply(pd.to_numeric, errors="coerce")

# 음수값 NaN 처리
neg_before = (num_targets < 0).sum().sum()
if NEGATIVE_TO_NAN:
    df_clean[target_cols] = num_targets.mask(num_targets < 0, np.nan)
else:
    df_clean[target_cols] = num_targets

# 대상 컬럼이 전부 NaN인 행 제거
rows_before_drop_empty = len(df_clean)
if DROP_EMPTY_TARGET:
    df_clean = df_clean.dropna(subset=target_cols, how="all").reset_index(drop=True)
rows_dropped_empty = rows_before_drop_empty - len(df_clean)

# IQR 이상치 제거 (대상 컬럼 기준, 어느 하나라도 범위 밖이면 제거)
rows_dropped_iqr = 0
if APPLY_IQR_FILTER:
    num_df = df_clean[target_cols].apply(pd.to_numeric, errors="coerce")
    Q1 = num_df.quantile(0.25)
    Q3 = num_df.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - IQR_K * IQR
    upper = Q3 + IQR_K * IQR

    outlier_low_mask  = num_df.lt(lower, axis=1)
    outlier_high_mask = num_df.gt(upper, axis=1)
    to_drop = (outlier_low_mask | outlier_high_mask).any(axis=1)

    rows_dropped_iqr = int(to_drop.sum())
    df_clean = df_clean.loc[~to_drop].reset_index(drop=True)

In [62]:
# 시간 컬럼 정렬
if "time_col" in globals() and time_col in df_clean.columns:
    df_clean[time_col] = pd.to_datetime(df_clean[time_col], errors="coerce")
    df_clean = df_clean.sort_values(time_col).reset_index(drop=True)

# 음수 잔존 여부 확인
neg_after = (df_clean[target_cols].apply(pd.to_numeric, errors="coerce") < 0).sum().sum()

print("=== Cleaning Summary ===")
print(f"Target columns (n={len(target_cols)}): {target_cols[:6]}{' ...' if len(target_cols)>6 else ''}")
print(f"Negatives (before -> after): {int(neg_before)} -> {int(neg_after)}")
print(f"IQR dropped rows (k={IQR_K}): {rows_dropped_iqr}")
print(f"Empty-target dropped rows: {rows_dropped_empty}")
print(f"Shape: raw {df_raw.shape} -> clean {df_clean.shape}")

=== Cleaning Summary ===
Target columns (n=8): ['Conc(ug/m3)', 'Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)'] ...
Negatives (before -> after): 0 -> 0
IQR dropped rows (k=1.5): 55
Empty-target dropped rows: 0
Shape: raw (517, 11) -> clean (461, 21)


In [64]:
import pandas as pd
import numpy as np

# ----- 설정: 이전 셀과 동일하게 맞추세요 -----
IQR_K = 1.5                # 이전 셀에서 사용한 값과 동일해야 함
NEGATIVE_TO_NAN = True     # 이전 셀과 동일
APPLY_IQR_FILTER = True    # 이전 셀과 동일
DROP_EMPTY_TARGET = True   # 이전 셀과 동일

SAVE_TO_EXCEL = False      # 제거된 행을 엑셀로 저장하려면 True

# ----- 전제: df_raw, df_clean, time_col, pm25_col, metal_cols 가 존재 -----
assert "df_raw" in globals(), "df_raw가 필요합니다. 앞선 셀을 먼저 실행하세요."
assert "df_clean" in globals(), "df_clean이 필요합니다. 앞선 전처리 셀을 먼저 실행하세요."

# 1) 대상 컬럼 재구성 (PM2.5 + 금속)
target_cols = []
if "pm25_col" in globals() and pm25_col:
    target_cols.append(pm25_col)
if "metal_cols" in globals() and isinstance(metal_cols, (list, tuple)):
    target_cols += list(metal_cols)
target_cols = [c for c in target_cols if c in df_raw.columns]
if len(target_cols) == 0:
    raise ValueError("전처리 대상 컬럼을 찾지 못했습니다. pm25_col/metal_cols를 확인하세요.")

# 2) 단계별 상태 재현: 음수→NaN 적용
num_targets_raw = df_raw[target_cols].apply(pd.to_numeric, errors="coerce")
stage1 = df_raw.copy()
stage1[target_cols] = num_targets_raw.mask(num_targets_raw < 0, np.nan) if NEGATIVE_TO_NAN else num_targets_raw

# 3) IQR 기준 위반 마스크 계산 (행 단위)
if APPLY_IQR_FILTER:
    num_df = stage1[target_cols].apply(pd.to_numeric, errors="coerce")
    Q1 = num_df.quantile(0.25)
    Q3 = num_df.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - IQR_K * IQR
    upper = Q3 + IQR_K * IQR

    mask_low  = num_df.lt(lower, axis=1)
    mask_high = num_df.gt(upper, axis=1)
    mask_iqr_row = (mask_low | mask_high).any(axis=1)
else:
    mask_low = pd.DataFrame(False, index=stage1.index, columns=target_cols)
    mask_high = pd.DataFrame(False, index=stage1.index, columns=target_cols)
    mask_iqr_row = pd.Series(False, index=stage1.index)

# 4) IQR로 제거된 행 상세 (어떤 컬럼이 위반인지 표시)
removed_iqr = stage1.loc[mask_iqr_row].copy()

def violating_cols(row_idx):
    cols = []
    for c in target_cols:
        if mask_low.at[row_idx, c] or mask_high.at[row_idx, c]:
            cols.append(c)
    return ", ".join(cols)

if not removed_iqr.empty:
    removed_iqr["_violating_cols"] = [violating_cols(i) for i in removed_iqr.index]

# 5) Empty-target으로 제거된 행(= 대상 컬럼이 모두 NaN인 행)
stage2 = stage1.loc[~mask_iqr_row].copy()
mask_empty = stage2[target_cols].isna().all(axis=1)
removed_empty = stage2.loc[mask_empty].copy()

# 6) 시간열이 있으면 앞단에 배치(가독성)
cols_to_show = [time_col] + target_cols if ("time_col" in globals() and time_col in stage1.columns) else target_cols
cols_to_show = [c for c in cols_to_show if c in stage1.columns]  # 존재하는 컬럼만

print("=== Removed by IQR (rows) ===")
print(f"count: {len(removed_iqr)}")
display_cols = cols_to_show + (["_violating_cols"] if "_violating_cols" in removed_iqr.columns else [])
display(removed_iqr[display_cols].head(10))  # 너무 길면 상위 10행만 표시

print("\n=== Removed by Empty-target (rows) ===")
print(f"count: {len(removed_empty)}")
if not removed_empty.empty:
    display(removed_empty[cols_to_show])

=== Removed by IQR (rows) ===
count: 55


,Pump-Begin,Conc(ug/m3),Cr(ng/m3),Co(ng/m3),Ni(ng/m3),As(ng/m3),Cd(ng/m3),Sb(ng/m3),Pb(ng/m3),_violating_cols
5,2025-01-10 16:00:00,8.44,5.97,8.4,7.15,0,310.06,60.2,3.83,Sb(ng/m3)
11,2025-01-10 22:00:01,5.04,7.3,8.59,8.66,0,378.53,62.37,6.23,"Ni(ng/m3), Cd(ng/m3), Sb(ng/m3)"
31,2025-01-11 18:00:01,8.96,4.55,10.12,5.61,0,291.87,55.86,2.77,Sb(ng/m3)
84,2025-01-13 23:00:00,21.82,3.97,9.27,3.89,0,243.27,66.62,4.57,Sb(ng/m3)
112,2025-01-15 03:00:00,18.95,4.64,20.28,3.85,0,254.34,6.56,15.13,Co(ng/m3)
121,2025-01-15 12:00:00,13.61,3.69,9.34,4.12,6.35,270.45,NaN,19.68,As(ng/m3)
122,2025-01-15 13:00:00,8.66,2.64,6.45,3,0.8,227.6,NaN,21.9,As(ng/m3)
163,2025-01-17 06:00:01,18.3,3.55,12.86,2.9,0,225.06,8.96,44.46,Pb(ng/m3)
180,2025-01-17 23:00:00,20.61,2.81,11.95,2.05,0.87,223.7,NaN,33.68,"As(ng/m3), Pb(ng/m3)"
181,2025-01-18 00:00:00,22.42,3.6,14.72,2.97,0,249.49,4.05,37.78,Pb(ng/m3)



=== Removed by Empty-target (rows) ===
count: 1


,Pump-Begin,Conc(ug/m3),Cr(ng/m3),Co(ng/m3),Ni(ng/m3),As(ng/m3),Cd(ng/m3),Sb(ng/m3),Pb(ng/m3)
0,2025-01-10 10:53:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 전처리 데이터 저장
df_clean.to_excel("Step1_cleaned.xlsx", index=False)
df_clean.to_csv("Step1_cleaned.csv", index=False, encoding="utf-8-sig")
print("Saved: Step1_cleaned.xlsx, Step1_cleaned.csv")